A short notebook where I show how to fine-tune the RoBERTa base model on the competition dataset using [transformers](https://huggingface.co/transformers/examples.html) CLI script. 
This would have been better on the [SquaD](https://rajpurkar.github.io/SQuAD-explorer/) dataset but due to lack of time, didn't try it. Also, notice that one of the versions is running using TPU. 
Enjoy!

In [ ]:
# Not necessary but you can get some logs.
!wandb login 22f6b417e27a2c1fd23e5d45d687926b3f9e3b85

Here, I start by loading the train and test, concatenating them and saving so that the file is available in a write folder. 
This is necssary for the CLI to work since accessing from the input folder alone fails (it is read-only). As I said, running on the SquaD dataset (or using the squad roberta base https://huggingface.co/deepset/roberta-base-squad2) would have been better I guess. 

In [ ]:
import pandas as pd


train_df = pd.read_csv("../input/tweet-sentiment-extraction/train.csv")
test_df = pd.read_csv("../input/tweet-sentiment-extraction/test.csv")

df = pd.concat([test_df, train_df])
df.to_csv("train_test.csv", index=False)

In [ ]:
# If you want TPU.
"""
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
!export XLA_USE_BF16=1
"""

So you need mainly need five things:
    
- where to store the fine-tuned model via the `output_dir` flag
- The model type via the `model_type` flag: here `roberta`
- The model base via the `model_name_or_path` flag: here `roberta-base` 
- The nature of the task via `--mlm` since it is masked language model here.
- The train input via `train_data_file` flag: here `train_test.csv`

In [ ]:
%%time
!python ../input/pytorchtransformers/examples/language-modeling/run_language_modeling.py \
--output_dir=fine_tuned_roberta_update                                                     \
--model_type=roberta                                                                 \
--model_name_or_path=roberta-base                                                     \
--do_train                                                                             \
--train_data_file=train_test.csv                                                             \
--mlm \
--num_train_epochs 5